In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df = pd.read_excel('/content/drive/MyDrive/TAMIL_AUDIO_PROJECT (File responses)/Rajalakshmi maam/Tamil-Codemixed_offensive_data_Training-Tweet-HL.xlsx')

In [8]:
df

,Unnamed: 0,texts,labels
0,TA_HL100,Iyaooo Kovam pattutene sothula visatha vachuru...,NOT
1,TA_HL101,@Asha Apo neenga atha government ku theriya pa...,NOT
2,TA_HL102,@Bala sundar ayyo sorry...antha line ah explai...,NOT
3,TA_HL105,@kalimuthu ne ena lusa...yaaru edhu panaalum e...,NOT
4,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT
...,...,...,...
3995,TA_TW6620,Yaroda body structure semaya irukum? Sema mood...,OFF
3996,TA_TW3336,Yenda naangala politics varom nu pala varusham...,OFF
3997,TA_HL1105,Yepdithan seruppala adichalum arasiyalvathikku...,OFF
3998,TA_TW1915,@USER Paithiyam ena unga vanthu full ah forwar...,OFF


In [9]:
df['labels'].unique()

array(['NOT', 'not', 'OFF', 'OFf'], dtype=object)

In [10]:
df["labels"] = df["labels"].str.lower()

In [11]:
df['labels'].unique()

array(['not', 'off'], dtype=object)

In [12]:
le = preprocessing.LabelEncoder()
le.fit(df['labels'])
df['labels'] = le.transform(df['labels'])

In [23]:
def clean_tweet(sentence):
    sentence = re.sub(r"https?://[A-Za-z0-9./]+",' ', sentence)
    #sentence = re.sub(regex_for_emojis,' ', sentence)
    sentence = re.sub("RT ", " ", sentence)
    sentence = re.sub("\n", " ", sentence)
    sentence = re.sub(r" +", " ", sentence)
    sentence = sentence.lower()
    sentence = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",sentence)
    sentence = re.sub("(\\d|\\W)+"," ",sentence)
    tokens = []
    for token in sentence.split():
        if token!="user":
            tokens.append(token)
    return " ".join(tokens)

In [24]:
cleaned_tweets = [clean_tweet(sentence) for sentence in df['texts']]

In [25]:
cleaned_tweets

['iyaooo kovam pattutene sothula visatha vachuruvanugale iooo',
 'asha apo neenga atha government ku theriya paduthaliya',
 'bala sundar ayyo sorry antha line ah explain pannama vittutu irukan drowsy ok na solran en appavum indha award function ku vanthurukkaru neenga en appava paakala pola en appavukku munnadiye ipdi enna insult panra maathri kevi kettu asinga paduthuringa nu solraaru yeah yeah chiiiii karumam podinnngggg asingama vaila vanthurum',
 'kalimuthu ne ena lusa yaaru edhu panaalum en da religion ah ilukreenga manishana paaru da modhalla',
 'st baby ku neat ah feed panunga plzz ipdi irundha epdi sapdum baby udambu full ah agitu plzz don t force tooo by doing this type of method',
 'il vazhgiromnu iruku ithula apdina',
 'varusa kadan age engayo idikuthe',
 'a vanitha veliya po ethuku thirumpi vantha',
 'à à à à à à à ˆ ga taste ah saptanum nu ninappanga nenga eappo pathalum paal satham paruppu neai nu thantha avanga saptave pidikkathu nenga mathi mathi tharanum',
 'aaiiii jol

In [33]:
vectorizer = TfidfVectorizer(min_df = 3)

In [34]:
X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()

In [35]:
len(X)

4000

In [36]:
y = df['labels']

In [37]:
len(y)

4000

### **Logistic regresssion**

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [40]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


LogisticRegression()

# **TESTING**

In [42]:
test_category = classifier.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [43]:
print(classification_report(y_test, test_category))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84       515
           1       0.85      0.79      0.82       485

    accuracy                           0.83      1000
   macro avg       0.83      0.83      0.83      1000
weighted avg       0.83      0.83      0.83      1000

